In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

client = QdrantClient(":memory:") # Para testar rápido na memória

client.create_collection(
    collection_name="meu_projeto_hibrido",
    # Configuração para busca por Significado (Densa)
    vectors_config=models.VectorParams(
        size=1536, # Ex: tamanho do embedding da OpenAI
        distance=models.Distance.COSINE
    ),
    # Configuração para busca por Palavra-Chave (Esparsa)
    sparse_vectors_config={
        "text-sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    }
)

In [ ]:
# Imagine que 'vetor_denso' e 'vetor_esparso' foram gerados pelos seus modelos
resultado = client.query(
    collection_name="meu_projeto_hibrido",
    prefetch=[
        # Busca 1: Pelo significado (Dense)
        models.Prefetch(
            query=vetor_denso,
            limit=20
        ),
        # Busca 2: Pela palavra exata (Sparse)
        models.Prefetch(
            query=vetor_esparso,
            using="text-sparse",
            limit=20
        ),
    ],
    # O "RRF" funde os dois rankings de forma inteligente
    query_filter=None, 
    ranking=models.Fusion.RRF, 
    limit=5
)